### 3. High-level analysis

Perform **at least 6** higher-level analyses of your data. You are free to use any techniques we discussed in class, including but not limited to:

* Use Pandas features to answer specific questions about the data
* Perform a cluster analysis to identify groups within your data
* Identify and motivate a machine learning problem in your data (classification or regression). Create a train/test/validation split and evaluate how well an appropriate model performs
* Perform a linear regression to show the relationship between two variables

If applicable to an analysis, you **must** include:

* Appropriate statistical test(s)
* An appropriate visualization.

Please take advantage of the check-ins or office hours if you are unsure whether a visualization or statistical test is necessary for an analysis.

In [12]:
import pandas as pd

df = pd.read_csv("../datasets/processed/arrests2025_cleaned.csv")
df.drop(columns=['New Georeferenced Column'], inplace=True)
df.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,...,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,MIN_AGE,MAX_AGE,DAY_OF_WEEK,MONTH,YEAR
0,298799078,2025-01-02 00:00:00.000000000,101,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,23,...,BLACK,1000213,228833,40.794755,-73.942348,25,44,Thursday,January,2025
1,299008265,2025-01-07 00:00:00.000000000,105,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,Q,113,...,BLACK,1046399,187126,40.680086,-73.775931,45,64,Tuesday,January,2025
2,298969999,2025-01-06 00:00:00.000000000,793,WEAPONS POSSESSION 3,118.0,DANGEROUS WEAPONS,PL 2650201,F,M,5,...,WHITE,983907,199958,40.715526,-74.001238,25,44,Monday,January,2025
3,299436365,2025-01-14 00:00:00.000000000,157,RAPE 1,104.0,RAPE,PL 130352B,F,Q,112,...,BLACK,1025401,202586,40.722641,-73.851542,45,64,Tuesday,January,2025
4,299562518,2025-01-16 00:00:00.000000000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601504,F,M,26,...,BLACK,996342,236149,40.814853,-73.956314,0,17,Thursday,January,2025
